In [5]:
%%time
import os
import subprocess
import datetime
import numpy as np
from sys import getsizeof
from IPython.display import Audio
import matplotlib.pyplot as plt

import torch
import torchaudio
from torchaudio.compliance import kaldi

print(torchaudio.__version__)

0.5.0a0+3305d5c
CPU times: user 3.84 ms, sys: 8.12 ms, total: 12 ms
Wall time: 16.7 ms


In [6]:
%%time
input_file = './example.wav'
resampled_file = './torch_resampled.wav'

wave, sr_original = torchaudio.load(input_file) 
# Note that this particular file has a single channel. Most audio files will have two (stereo) channels.

print('Data:', wave)
print('Sampling rate:', sr_original)
print('Samples:', len(wave))
print('Audio length:', round(wave.size()[1]/sr_original, 4), 'seconds')
print('Lowest amplitude:', wave.max().item())
print('Highest amplitude:', wave.min().item())

Audio(input_file)

Data: tensor([[-0.0516, -0.0500, -0.0494,  ...,  0.0178,  0.0198,  0.0197]])
Sampling rate: 44100
Samples: 1
Audio length: 4.5273 seconds
Lowest amplitude: 0.154296875
Highest amplitude: -0.164215087890625
CPU times: user 62.7 ms, sys: 15.3 ms, total: 78 ms
Wall time: 52.2 ms


In [7]:
# Let's fix target parameters:

seconds_target = 5.0
sr_target = 48000

In [9]:
%%time

#device = torch.device('cuda:0')
device = torch.device('cpu')
wave.to(device)
resampled_wave = torchaudio.compliance.kaldi.resample_waveform(wave, wave.size(1), seconds_target*sr_target)

CPU times: user 1min 6s, sys: 4.47 s, total: 1min 10s
Wall time: 17.7 s


In [10]:
# At CPU:
# CPU times: user 1min 9s, sys: 4.58 s, total: 1min 13s
# Wall time: 18.4 s
#
# At GPU:
# CPU times: user 1min 8s, sys: 4.99 s, total: 1min 13s
# Wall time: 19.6 s

In [16]:
#scaled_wave = (resampled_wave.numpy()/np.max(np.abs(resampled_wave.numpy())) * 32767)
torchaudio.save(resampled_file, torch.clamp(resampled_wave, -1, 1), sr_target, precision=32)
os.chown(resampled_file, 1000, 1000)
print('Data:', resampled_wave)
print('Sampling rate:', sr_target)
print('Samples:', len(resampled_wave))
print('Audio length:', round(resampled_wave.size(1)/sr_target, 4), 'seconds')
print('Lowest amplitude:', resampled_wave.min().item())
print('Highest amplitude:', resampled_wave.max().item())

Audio(resampled_file)

Data: tensor([[-0.0513, -0.0530, -0.0472,  ...,  0.0180,  0.0218,  0.0169]])
Sampling rate: 48000
Samples: 1
Audio length: 5.0 seconds
Lowest amplitude: -0.16443273425102234
Highest amplitude: 0.1545400321483612


In [17]:
torch.cuda.empty_cache()